### Baseline model
hidden_layer_sizes=(256, 128),   # 2층 MLP <br> activation='relu', <br> solver='adam', <br> alpha=1e-4, <br> batch_size=128, <br> learning_rate='adaptive', <br> learning_rate_init=1e-3, <br> max_iter=200, <br> early_stopping=True

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_auc_score, precision_recall_curve, auc

In [11]:
train_df = pd.read_csv("../../../dataset/open_world/openworld_train.csv")
test_df = pd.read_csv("../../../dataset/open_world/openworld_test.csv")

print("[INFO] Train shape:", train_df.shape)
print("[INFO] Test shape:", test_df.shape)

[INFO] Train shape: (20300, 27)
[INFO] Test shape: (8700, 27)


In [12]:
# feature/label 분리
target_col = train_df.columns[-1]   # 마지막 컬럼이 label

# open-world label 통합
train_df[target_col] = train_df[target_col].apply(lambda x: 0 if x == 95 else 1)
test_df[target_col]  = test_df[target_col].apply(lambda x: 0 if x == 95 else 1)

X_train = train_df.drop(columns=[target_col]).values
y_train = train_df[target_col].astype(np.float32).values
X_test  = test_df.drop(columns=[target_col]).values
y_test  = test_df[target_col].astype(np.float32).values

# train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

print("[INFO] X_train:", X_train.shape, " | X_val:", X_val.shape, " | X_test:", X_test.shape)

[INFO] X_train: (16240, 26)  | X_val: (4060, 26)  | X_test: (8700, 26)


In [13]:
mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128),   # 2층 MLP
    activation='relu',
    solver='adam',
    alpha=1e-4,                     # L2 regularization
    batch_size=128,
    learning_rate='adaptive',
    learning_rate_init=1e-3,
    max_iter=200,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42,
    verbose=True
)

In [14]:
print("\n[INFO] Training MLP (Binary) baseline model ...")
mlp.fit(X_train, y_train)


[INFO] Training MLP (Binary) baseline model ...
Iteration 1, loss = 0.48079609
Validation score: 0.805419
Iteration 2, loss = 0.41007542
Validation score: 0.821429
Iteration 3, loss = 0.38685796
Validation score: 0.828818
Iteration 4, loss = 0.37321980
Validation score: 0.830665
Iteration 5, loss = 0.35999698
Validation score: 0.833744
Iteration 6, loss = 0.35370699
Validation score: 0.837438
Iteration 7, loss = 0.34492065
Validation score: 0.844828
Iteration 8, loss = 0.33688614
Validation score: 0.841749
Iteration 9, loss = 0.32943964
Validation score: 0.853448
Iteration 10, loss = 0.32448988
Validation score: 0.838670
Iteration 11, loss = 0.31698905
Validation score: 0.844212
Iteration 12, loss = 0.31408893
Validation score: 0.858374
Iteration 13, loss = 0.30477262
Validation score: 0.847906
Iteration 14, loss = 0.30051174
Validation score: 0.854064
Iteration 15, loss = 0.29500872
Validation score: 0.857143
Iteration 16, loss = 0.28466149
Validation score: 0.849138
Iteration 17, lo

MLPClassifier(batch_size=128, early_stopping=True,
              hidden_layer_sizes=(256, 128), learning_rate='adaptive',
              random_state=42, verbose=True)

In [15]:
val_pred = mlp.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
val_f1_macro = f1_score(y_val, val_pred, average='macro')
val_f1_weighted = f1_score(y_val, val_pred, average='weighted')

print("\n[VAL RESULTS]")
print(f"Accuracy     : {val_acc:.4f}")
print(f"F1 (macro)   : {val_f1_macro:.4f}")
print(f"F1 (weighted): {val_f1_weighted:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, val_pred, digits=4))


[VAL RESULTS]
Accuracy     : 0.8603
F1 (macro)   : 0.8410
F1 (weighted): 0.8582

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8351    0.7414    0.7855      1400
         1.0     0.8715    0.9229    0.8965      2660

    accuracy                         0.8603      4060
   macro avg     0.8533    0.8322    0.8410      4060
weighted avg     0.8589    0.8603    0.8582      4060



In [16]:
test_pred = mlp.predict(X_test)
test_pred_proba = mlp.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, test_pred)
test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

# ROC-AUC (binary)
test_roc_auc = roc_auc_score(y_test, test_pred_proba)

# PR-AUC
prec, rec, _ = precision_recall_curve(y_test, test_pred_proba)
test_pr_auc = auc(rec, prec)

print("\n========== [TEST RESULTS] ==========")
print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC         : {test_roc_auc:.4f}")
print(f"PR-AUC          : {test_pr_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_pred, digits=4))


========== [TEST RESULTS] ==========
Accuracy        : 0.8539
F1 (macro)      : 0.8333
F1 (micro)      : 0.8539
F1 (weighted)   : 0.8515
ROC-AUC         : 0.9227
PR-AUC          : 0.9558

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8271    0.7287    0.7748      3000
         1.0     0.8656    0.9198    0.8919      5700

    accuracy                         0.8539      8700
   macro avg     0.8464    0.8242    0.8333      8700
weighted avg     0.8523    0.8539    0.8515      8700

